## Deliverable 2. Create a Customer Travel Destinations Map.

In [33]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [34]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Lincoln,AU,-34.7333,135.8667,54.75,74,100,15.66,overcast clouds
1,1,Port Alfred,ZA,-33.5906,26.8910,63.18,73,0,5.01,clear sky
2,2,College,US,64.8569,-147.8028,55.22,79,75,0.00,broken clouds
3,3,Saint-Augustin,CA,51.2260,-58.6502,72.59,52,8,14.92,clear sky
4,4,Marawi,PH,7.9986,124.2928,69.89,95,100,1.83,overcast clouds


In [35]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip?  75
What is the maximum temperature you would like for your trip?  80


In [36]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
13,13,Pathein,MM,16.7833,94.7333,77.58,92,100,10.16,overcast clouds
15,15,Meulaboh,ID,4.1363,96.1285,79.74,79,100,5.06,overcast clouds
23,23,Tura,IN,25.5198,90.2201,76.37,95,100,3.40,overcast clouds
24,24,Atuona,PF,-9.8000,-139.0333,77.00,76,12,14.58,few clouds
31,31,Noumea,NC,-22.2763,166.4572,75.99,99,8,9.71,clear sky
49,49,Alofi,NU,-19.0595,-169.9187,77.07,68,9,18.41,clear sky
51,51,Qingdao,CN,36.0986,120.3719,76.82,84,0,4.07,clear sky
58,58,Faanui,PF,-16.4833,-151.7500,76.82,74,4,13.65,light rain
61,61,Bengkulu,ID,-3.8004,102.2655,77.22,89,80,4.50,broken clouds
65,65,Kandrian,PG,-6.2167,149.5500,76.64,89,15,3.80,light rain


In [37]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [38]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_no_nulls_df = preferred_cities_df.dropna() 

In [39]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_no_nulls_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Pathein,MM,77.58,overcast clouds,16.7833,94.7333,
15,Meulaboh,ID,79.74,overcast clouds,4.1363,96.1285,
23,Tura,IN,76.37,overcast clouds,25.5198,90.2201,
24,Atuona,PF,77.00,few clouds,-9.8000,-139.0333,
31,Noumea,NC,75.99,clear sky,-22.2763,166.4572,
49,Alofi,NU,77.07,clear sky,-19.0595,-169.9187,
51,Qingdao,CN,76.82,clear sky,36.0986,120.3719,
58,Faanui,PF,76.82,light rain,-16.4833,-151.7500,
61,Bengkulu,ID,77.22,broken clouds,-3.8004,102.2655,
65,Kandrian,PG,76.64,light rain,-6.2167,149.5500,


In [40]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    try:
    
        # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping ")
        
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
-----------------------------
Data Retrieval Complete      
-----------------------------


In [57]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Pathein,MM,77.58,overcast clouds,16.7833,94.7333,Pathein Hotel
15,Meulaboh,ID,79.74,overcast clouds,4.1363,96.1285,Kost Pria
23,Tura,IN,76.37,overcast clouds,25.5198,90.2201,Hotel RIKMAN Continental
24,Atuona,PF,77.00,few clouds,-9.8000,-139.0333,Villa Enata
31,Noumea,NC,75.99,clear sky,-22.2763,166.4572,Hôtel Le Lagon
...,...,...,...,...,...,...,...
653,Heide,DE,75.18,overcast clouds,54.2000,9.1000,nordica Hotel Berlin
656,Myanaung,MM,79.68,overcast clouds,18.2833,95.3167,Taung Hlay Khar
657,Boa Vista,BR,78.78,broken clouds,2.8197,-60.6733,Aipana Plaza Hotel
658,Alakurtti,RU,77.79,overcast clouds,66.9672,30.3491,Agroturystyka Gientosław


In [58]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [59]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [60]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))